### To Do
* update for additional nested metadata fields
* create that building block - get API metadata on each of the partitions so that its `python api.py <filename>_publications.json` 
* then second step is below - stacking, deduplication and formatting
* `python test.py` should be the final step

In [1]:
import os

#### Import and stack the partitions

In [2]:
partitions = [os.path.join('/Users/sophierand/RCPublications/partitions',f) for f in os.listdir('/Users/sophierand/RCPublications/partitions') if f.endswith('.json')]
# partitions                                               

In [3]:
import json
pub_list = []
for p in partitions:
    with open(p) as json_file:
        partition = json.load(json_file)
        pub_list.append(partition)

In [4]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [5]:
pub_list_flat = flatten(pub_list)

In [23]:
import collections
from collections import defaultdict
import pandas as pd

def combine_metadata(dics):
    dd = defaultdict(list)
    for dic in dics:
        for key, val in dic.items():  
            if not pd.isnull(val):
                dd[key].append(val)
#                 dd[key].append(val.lower())
                dd[key] = list(set(dd[key]))
            else: 
                continue
    return dict(dd)

def merge_metadata(dicts):
    res = collections.defaultdict(list)
    md_list = []
    for d in dicts:
        for k, v in d.items():
            if k  == 'original':
                md_list.append(v)
            if k == 'title':
                res[k].append(v)
            if k == 'datasets':
                res[k].append(v)
    res['original']  = combine_metadata(md_list)
    res['title'] = list(set(res['title']))[0]
    res['datasets'] = list(set(flatten(res['datasets'])))
    return dict(res)

#### Account for duplicates, merging any metadata into one subdictionary ('original')

In [24]:
from collections import Counter
ct = Counter(i['title'] for i in pub_list_flat)

In [25]:
new_l = []
for i,v in ct.items():
    if v > 1:
        dup_pubs = [p for p in pub_list_flat if p['title'] == i]
        dup_pubs_merged = merge_metadata(dup_pubs)
        new_l.append(dup_pubs_merged)
    if v == 1:
        unique_pubs = [p for p in pub_list_flat if p['title'] == i]
        new_l.append(unique_pubs)

In [9]:
json_pub_path = '/Users/sophierand/RCPublications/publications.json'

In [10]:
with open(json_pub_path, 'w') as outfile:
    json.dump(new_l, outfile, indent=2)